#Results

The data set is about German credit card data. 

Goal is to predict if the customer is good or bad in terms of repaying the credit taken from the bank.

It is worse to class a customer as good when they are bad, than it is to class a customer as bad when they are good.

Hence, minority class is the class of focus, I chose F2 measure as the metric.


In [ ]:
!pip install feature-engine

In [ ]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier

In [ ]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

pd.pandas.set_option('display.max_columns', None)
%matplotlib inline

#Exploratory analysis

In [ ]:
data = pd.read_csv('./german_syn.data',delim_whitespace=True)
copy = data.drop(columns='target')

In [ ]:
data.describe()

,v2,v5,v8,v11,v13,v16,v18,target
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,20.903000,3271.258000,2.973000,2.845000,35.546000,1.407000,1.155000,1.300000
std,12.058814,2822.736876,1.118715,1.103718,11.375469,0.577654,0.362086,0.458487
min,4.000000,250.000000,1.000000,1.000000,19.000000,1.000000,1.000000,1.000000
25%,12.000000,1365.500000,2.000000,2.000000,27.000000,1.000000,1.000000,1.000000
50%,18.000000,2319.500000,3.000000,3.000000,33.000000,1.000000,1.000000,1.000000
75%,24.000000,3972.250000,4.000000,4.000000,42.000000,2.000000,1.000000,2.000000
max,72.000000,18424.000000,4.000000,4.000000,75.000000,4.000000,2.000000,2.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      1000 non-null   object
 1   v2      1000 non-null   int64 
 2   v3      998 non-null    object
 3   v4      998 non-null    object
 4   v5      1000 non-null   int64 
 5   v6      1000 non-null   object
 6   v7      1000 non-null   object
 7   v8      1000 non-null   int64 
 8   v9      998 non-null    object
 9   v10     981 non-null    object
 10  v11     1000 non-null   int64 
 11  v12     994 non-null    object
 12  v13     1000 non-null   int64 
 13  v14     988 non-null    object
 14  v15     1000 non-null   object
 15  v16     1000 non-null   int64 
 16  v17     987 non-null    object
 17  v18     1000 non-null   int64 
 18  v19     1000 non-null   object
 19  v20     1000 non-null   object
 20  target  1000 non-null   int64 
dtypes: int64(8), object(13)
memory usage: 164.2+ KB


We see that there are approximately 5.6% missing values in the dataset. Let's first find out the types of variables that we have, and based on that we will decide our method to deal with these missing values.

In [ ]:
#let's determine the different types of variables in our dataset
categorical = [var for var in data.columns if data[var].dtype == 'O']

temporal = [var for var in data.columns if 'Yr' in var or 'Year' in var]

discrete = [
    var for var in data.columns if data[var].dtype != 'O'
    and len(data[var].unique()) < 10 and var not in temporal
]

continuous = [
    var for var in data.columns if data[var].dtype != 'O'
    if var not in discrete and var not in ['target']
    and var not in temporal
]

In [ ]:
temporal

[]

In [ ]:
categorical

['v1',
 'v3',
 'v4',
 'v6',
 'v7',
 'v9',
 'v10',
 'v12',
 'v14',
 'v15',
 'v17',
 'v19',
 'v20']

In [ ]:
discrete

['v8', 'v11', 'v16', 'v18', 'target']

In [ ]:
continuous

['v2', 'v5', 'v13']

There are 13 categorical variables, and 7 numerical variables in the dataset. All the variables that have missing values are from the set of categorical variables.

In [ ]:
data.target.value_counts()

1    700
2    300
Name: target, dtype: int64

The count of unique values in our target variable shows that the dataset is imbalanced with 70% observations falling in the majority class (target=1) and 30% minority class  (target=2).

#Splitting into train and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['target'], axis=1),
                                                    data['target'],
                                                    test_size=0.1,
                                                    random_state=0)

In [ ]:
X_train.shape

(900, 20)

In [ ]:
X_test.shape

(100, 20)

In [ ]:
discrete = ['v8', 'v11', 'v16', 'v18']
X_train[discrete] = X_train[discrete].astype('O')
X_test[discrete] = X_test[discrete].astype('O')

#Missing values imputation

In [ ]:
categorical = discrete +categorical

In [ ]:
categorical

['v8',
 'v11',
 'v16',
 'v18',
 'v1',
 'v3',
 'v4',
 'v6',
 'v7',
 'v9',
 'v10',
 'v12',
 'v14',
 'v15',
 'v17',
 'v19',
 'v20']

In [ ]:
mode_freq_columns= [var for var in categorical if (data[var].isnull().mean()<0.06 
                                                              and data[var].isnull().mean()>0)]

In [ ]:
mode_freq_columns

['v3', 'v4', 'v9', 'v10', 'v12', 'v14', 'v17']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import feature_engine.missing_data_imputers as mdi
import feature_engine.categorical_encoders as ce
from feature_engine.variable_transformers import YeoJohnsonTransformer
from sklearn.preprocessing import StandardScaler
from feature_engine.discretisers import DecisionTreeDiscretiser
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.categorical_encoders import OneHotCategoricalEncoder

I am one hot encoding all categorical variables, and scaling all the continuous ones. Also, imputing missing values with the ost frequent category.

In [ ]:
preprocessor = Pipeline([    
    ('imputer_cat_freq',
     mdi.CategoricalVariableImputer(imputation_method='frequent',variables=mode_freq_columns)),

    # categorical encoding 
    
    ('categorical_encoder',
     OneHotCategoricalEncoder( top_categories=None,
                               variables=categorical, # we can select which variables to encode
                               drop_last=True)),
    ('scaler', SklearnTransformerWrapper(transformer=StandardScaler(), variables = continuous))

])

In [ ]:
# now we fit the preprocessor
preprocessor.fit(X_train)

Pipeline(memory=None,
         steps=[('imputer_cat_freq',
                 CategoricalVariableImputer(fill_value='Missing',
                                            imputation_method='frequent',
                                            return_object=False,
                                            variables=['v3', 'v4', 'v9', 'v10',
                                                       'v12', 'v14', 'v17'])),
                ('categorical_encoder',
                 OneHotCategoricalEncoder(drop_last=True, top_categories=None,
                                          variables=['v8', 'v11', 'v16', 'v18',
                                                     'v1', 'v3', 'v4', 'v6',
                                                     'v7', 'v9', 'v10', 'v12',
                                                     'v14', 'v15', 'v17', 'v19',
                                                     'v20'])),
                ('scaler',
                 SklearnTransformerWrapper(transfor

In [ ]:
# and now we can impute the data

X_train = preprocessor.transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
y_train

785    1
873    1
65     1
902    1
317    1
      ..
835    2
192    2
629    1
559    2
684    1
Name: target, Length: 900, dtype: int64

Labeling the target variable to 0s and 1s instead of 1s and 2s.

In [ ]:
from sklearn.preprocessing import LabelEncoder
y_train=LabelEncoder().fit_transform(y_train)

In [ ]:
y_test=LabelEncoder().fit_transform(y_test)

In [ ]:
y_train

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,

#F2 score metric

We will be using F2 score as the metric in our problem, because our focus is on minimizing False Negatives. It is worse to class a customer as good when they are bad, than it is to classify a customer as bad when they are good.

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.metrics import fbeta_score

In [ ]:
def f2_measure(y_true, y_pred):
    return fbeta_score(y_true, y_pred, beta=2)

In [ ]:
metric = make_scorer(f2_measure)

In [ ]:
def evaluate_model(X, y, model):
	# define evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# define the model evaluation the metric
	metric = make_scorer(f2_measure)
	# evaluate model
	scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
	return scores

#Dummy Classifier

In [ ]:
from sklearn.dummy import DummyClassifier
from numpy import std
dummy_model = DummyClassifier(strategy='constant',constant=1)
scores = evaluate_model(X_train, y_train, dummy_model)
# summarize performance
print('Mean F2: %.3f (%.3f)' % (mean(scores), std(scores)))
#dummy_majority.fit(X_train, y_train)

#pred_most_frequent = dummy_majority.predict(X_test)

#print("Unique predicted labels: {}".format(np.unique(pred_most_frequent)))
#print("Test score: {:.2f}".format(dummy_majority.score(X_test, y_test)))

Mean F2: 0.687 (0.006)


From the performance of the dummy classifier, we can say that the baseline for evaluation of our further models will be a f2 score of 0.687. Our other models must perform better than this.

#Cost sensitive algorithms

##Logistic regression

In [ ]:
logreg = LogisticRegression(solver='lbfgs',max_iter=1000)
param_grid = {
    
    'class_weight': [{0:70,1:30}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}],
}

#apply grid search
grid_logreg= GridSearchCV(logreg, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_logreg.fit(X_train, y_train)

print("Best parameters: {}".format(grid_logreg.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_logreg.best_score_))
print('Train score: ', grid_logreg.score(X_train, y_train))
print('Test score: ', grid_logreg.score(X_test, y_test))

Best parameters: {'class_weight': {0: 1, 1: 10}}
Best Mean cross-validation score: 0.72
Train score:  0.7730167921250723
Test score:  0.5780346820809248


## Cost Sensitive SVM

In [ ]:
svc = SVC()
param_grid = {
    
    'class_weight': [{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}],
}

#apply grid search
grid_svc= GridSearchCV(svc, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_svc.fit(X_train, y_train)

print("Best parameters: {}".format(grid_svc.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_svc.best_score_))
print('Train score: ', grid_svc.score(X_train, y_train))
print('Test score: ', grid_svc.score(X_test, y_test))

Best parameters: {'class_weight': {0: 1, 1: 100}}
Best Mean cross-validation score: 0.67
Train score:  0.8957654723127035
Test score:  0.6493506493506492


## Decison Tree

In [ ]:
dtree = DecisionTreeClassifier()
param_grid = {
    
    'class_weight': [{0:100,1:1}, {0:10,1:1}, {0:1,1:1}, {0:1,1:10},{0:1,1:100}],
}

#apply grid search
grid_dtree= GridSearchCV(dtree, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_dtree.fit(X_train, y_train)

print("Best parameters: {}".format(grid_dtree.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_dtree.best_score_))
print('Train score: ', grid_dtree.score(X_train, y_train))
print('Test score: ', grid_dtree.score(X_test, y_test))

Best parameters: {'class_weight': {0: 1, 1: 1}}
Best Mean cross-validation score: 0.54
Train score:  1.0
Test score:  0.38167938931297707


##Gradient Boosting with XGBoost

In [ ]:
xgboost = XGBClassifier()
param_grid = {
    
    'scale_pos_weight': [1,2, 10, 25, 50],
}

#apply grid search
grid_xgboost= GridSearchCV(xgboost, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_xgboost.fit(X_train, y_train)

print("Best parameters: {}".format(grid_xgboost.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_xgboost.best_score_))
print('Train score: ', grid_xgboost.score(X_train, y_train))
print('Test score: ', grid_xgboost.score(X_test, y_test))

Best parameters: {'scale_pos_weight': 25}
Best Mean cross-validation score: 0.73
Train score:  0.793879907621247
Test score:  0.625


##Extra trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

extra_model = ExtraTreesClassifier(n_estimators=10, class_weight='balanced')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(extra_model, X_train, y_train, scoring=metric, cv=cv, n_jobs=-1)
extra_model.fit(X_train,y_train)
# summarize performance
print('Mean f2 score: %.3f' % mean(scores))
print('Train score: ', extra_model.score(X_train, y_train))
print('Test score: ', extra_model.score(X_test, y_test))

Mean f2 score: 0.384
Train score:  1.0
Test score:  0.77


##Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10, class_weight='balanced')
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring=metric, cv=cv, n_jobs=-1)
model.fit(X_train,y_train)
# summarize performance
print('Mean f2 score: %.3f' % mean(scores))
print('Train score: ', model.score(X_train, y_train))
print('Test score: ', model.score(X_test, y_test))

Mean f2 score: 0.380
Train score:  0.9755555555555555
Test score:  0.75


##Bagging Decision tree with undersampling

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
# define model
model = BalancedBaggingClassifier()
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring=metric, cv=cv, n_jobs=-1)
model.fit(X_train,y_train)
# summarize performance
print('Mean f2 score: %.3f' % mean(scores))
print('Train score: ', model.score(X_train, y_train))
print('Test score: ', model.score(X_test, y_test))

Mean f2 score: 0.585
Train score:  0.9611111111111111
Test score:  0.73


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

##Stacking classifier for cost sensitive algorithm

In [ ]:
from sklearn.ensemble import StackingClassifier
sclf1 = StackingClassifier(estimators=
                              [('xgbc',grid_xgboost.best_estimator_)
                               
                              ], final_estimator=LogisticRegression())
sclf1_param = {
              'final_estimator__C' : [0.5,0.7,1]
             }
sclf1_grid = GridSearchCV(sclf1, sclf1_param,cv=5, return_train_score=True,scoring= metric  )
sclf1_grid.fit(X_train_1,y_train)
print(f'Best Mean Cross Validation Score is {sclf1_grid.best_score_}')
print(f'Best Mean Cross Validation Score is {sclf1_grid.best_params_}')
print(f'Train score is {sclf1_grid.score(X_train_1,y_train)}')
print(f'Test score is {sclf1_grid.score(X_test_1,y_test)}')

Best Mean Cross Validation Score is 0.209565913216284
Best Mean Cross Validation Score is {'final_estimator__C': 1}
Train score is 0.5161290322580645
Test score is 0.08928571428571429


#Data Sampling Algorithms

In [ ]:
from imblearn.pipeline import Pipeline as imb_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN

##Logistic regression

In [ ]:
pipe_rand_smote = imb_pipeline([('smote', SMOTE()), ('model', LogisticRegression())])
param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5],
    'model__penalty': ['l1','l2'],
}

#apply grid search
grid_smote= GridSearchCV(pipe_rand_smote, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_smote.fit(X_train, y_train)

print("Best parameters: {}".format(grid_smote.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_smote.best_score_))
print('Train score: ', grid_smote.score(X_train, y_train))
print('Test score: ', grid_smote.score(X_test, y_test))

Best parameters: {'model__penalty': 'l2', 'smote__k_neighbors': 4}
Best Mean cross-validation score: 0.66
Train score:  0.7142857142857142
Test score:  0.5714285714285714


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


##Decision tree

In [ ]:
pipe_rand_smote = imb_pipeline([('smote', SMOTE()), ('model', DecisionTreeClassifier())])
param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5],
    'model__max_depth': [2,3,4,5,6],
    'model__splitter': ['random']
}

#apply grid search
grid_smote= GridSearchCV(pipe_rand_smote, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_smote.fit(X_train, y_train)

print("Best parameters: {}".format(grid_smote.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_smote.best_score_))
print('Train score: ', grid_smote.score(X_train, y_train))
print('Test score: ', grid_smote.score(X_test, y_test))

Best parameters: {'model__max_depth': 2, 'model__splitter': 'random', 'smote__k_neighbors': 3}
Best Mean cross-validation score: 0.64
Train score:  0.6020408163265307
Test score:  0.49295774647887325


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


##KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
pipe_rand_smote = imb_pipeline([('smote', SMOTE()), ('model', KNeighborsClassifier())])
param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5],
    'model__n_neighbors': range(10,20),
    'model__algorithm':['ball_tree', 'kd_tree', 'brute']
}

#apply grid search
grid_smote= GridSearchCV(pipe_rand_smote, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_smote.fit(X_train, y_train)

print("Best parameters: {}".format(grid_smote.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_smote.best_score_))
print('Train score: ', grid_smote.score(X_train, y_train))
print('Test score: ', grid_smote.score(X_test, y_test))

Best parameters: {'model__algorithm': 'ball_tree', 'model__n_neighbors': 19, 'smote__k_neighbors': 4}
Best Mean cross-validation score: 0.71
Train score:  0.7752179327521791
Test score:  0.5864197530864198


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


##SVM

In [ ]:
pipe_rand_smote = imb_pipeline([('smote', SMOTE()), ('model', SVC())])
param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5]
}

#apply grid search
grid_smote= GridSearchCV(pipe_rand_smote, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_smote.fit(X_train, y_train)

print("Best parameters: {}".format(grid_smote.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_smote.best_score_))
print('Train score: ', grid_smote.score(X_train, y_train))
print('Test score: ', grid_smote.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Best parameters: {'smote__k_neighbors': 3}
Best Mean cross-validation score: 0.61
Train score:  0.8475783475783476
Test score:  0.5725190839694656


##Random Forest

In [ ]:
pipe_rand_smote = imb_pipeline([('smote', SMOTE()), ('model', RandomForestClassifier())])
param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5]
}

#apply grid search
grid_smote= GridSearchCV(pipe_rand_smote, param_grid, cv=5, n_jobs=2, scoring=metric)
grid_smote.fit(X_train, y_train)

print("Best parameters: {}".format(grid_smote.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_smote.best_score_))
print('Train score: ', grid_smote.score(X_train, y_train))
print('Test score: ', grid_smote.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Best parameters: {'smote__k_neighbors': 3}
Best Mean cross-validation score: 0.51
Train score:  1.0
Test score:  0.4365079365079365


##Easy Ensemble classifier

In [ ]:
from imblearn.ensemble import EasyEnsembleClassifier

# define model
model = EasyEnsembleClassifier(n_estimators=10)
# define evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring=metric, cv=5, n_jobs=-1)
model.fit(X_train, y_train)
# summarize performance
print('Mean f2 score: %.3f' % mean(scores))
print('Train score: ', model.score(X_train, y_train))
print('Test score: ', model.score(X_test, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Mean f2 score: 0.667
Train score:  0.7622222222222222
Test score:  0.71


##XGBoost

In [ ]:
X_test_1 = X_test.to_numpy()
X_train_1 = X_train.to_numpy()

In [ ]:
from  xgboost import XGBClassifier
xgbc= XGBClassifier(random_state=42,early_stopping_rounds=2,objective= 'binary:logistic')
xgbc_us_2 = imb_pipeline([('smote',SMOTE()),('model',xgbc)])
xgbc_us_2_param = {
              'model__max_depth' : [2,4,6],
              'model__subsample':[0.6,0.8,1],
              "smote__k_neighbors":[4,12]
             }
xgbc_us_2_grid = GridSearchCV(xgbc_us_2, xgbc_us_2_param,cv=5, return_train_score=True,scoring= metric )
xgbc_us_2_grid.fit(X_train_1,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('smote',
                                        SMOTE(k_neighbors=5, kind='deprecated',
                                              m_neighbors='deprecated',
                                              n_jobs=1, out_step='deprecated',
                                              random_state=None, ratio=None,
                                              sampling_strategy='auto',
                                              svm_estimator='deprecated')),
                                       ('model',
                                        XGBClassifier(base_score=0.5,
                                                      booster='gbtree',
                                                      colsample_bylevel=1,
                                                      colsample_bynode=...
                                                      random_state=42,
  

In [ ]:
print('best parameters: ',xgbc_us_2_grid.best_params_)
print('Mean f2 score: %.3f' % mean(scores))
print('Train score: ', xgbc_us_2_grid.score(X_train_1, y_train))
print('Test score: ', xgbc_us_2_grid.score(X_test_1, y_test))

best parameters:  {'model__max_depth': 2, 'model__subsample': 0.6, 'smote__k_neighbors': 4}
Mean f2 score: 0.668
Train score:  0.6981818181818183
Test score:  0.6060606060606061


##Stacking classifiers

In [ ]:
from sklearn.ensemble import StackingClassifier
sclf1 = StackingClassifier(estimators=
                              [('xgbc',xgbc_us_2_grid.best_estimator_), 
                               ('eec',EasyEnsembleClassifier(n_estimators=10))
                              ], final_estimator=LogisticRegression())
sclf1_param = {
              'final_estimator__C' : [0.5,0.7,1]
             }
sclf1_grid = GridSearchCV(sclf1, sclf1_param,cv=5, return_train_score=True,scoring= metric  )
sclf1_grid.fit(X_train_1,y_train)
print(f'Best Mean Cross Validation Score is {sclf1_grid.best_score_}')
print(f'Best Mean Cross Validation Score is {sclf1_grid.best_params_}')
print(f'Train score is {sclf1_grid.score(X_train_1,y_train)}')
print(f'Test score is {sclf1_grid.score(X_test_1,y_test)}')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Best Mean Cross Validation Score is 0.48550034049608815
Best Mean Cross Validation Score is {'final_estimator__C': 0.7}
Train score is 0.5852713178294574
Test score is 0.4435483870967742


#Stacking Classifiers

In [ ]:
layer_one_estimators = [
                        ('xgbc',xgbc_us_2_grid.best_estimator_)            
                       ]
layer_two_estimators = [
                        ('eec',EasyEnsembleClassifier(n_estimators=10))
                       ]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

# Create Final model by 
clf = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(clf, X_train_1, y_train, scoring=metric, cv=cv, n_jobs=-1)
clf.fit(X_train_1, y_train)
# summarize performance
print('Mean f2 score: %.3f' % mean(scores))
print(f'Train score is {clf.score(X_train_1,y_train)}')
print(f'Test score is {clf.score(X_test_1,y_test)}')

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

Mean f2 score: 0.000
Train score is 0.6944444444444444
Test score is 0.75


#Final predictions

In [ ]:
xgbc_us_2_grid.predict(X_test_1)

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0])

In [ ]:
xgbc_us_2_grid.score(X_test_1)